In [1]:
import torch

# 追踪梯度
# pytorch 通过提供包(package)autograde, 计算函数梯度
# Tensor 是这个包的核心类
# .require_grad 设为True, 追踪tensor上的所有操作，可以使用链式法则进行梯度传播
# .backword() 完成所有的梯度计算，
# .grad 累积梯度
#  取消梯度追踪
# .detach() 将tensor从追踪记录中分离出来， 可以防止被计算追踪，梯度就传不过去
# 或者使用 with torch.no_grad() 将不想被追踪的操作代码块包裹起来（如模型评估时，不需要计算参数梯度）

# Function 是另一个类，Tensor和Function互相结合就可以构建一个记录整个计算过程的有向無环图（DAG）
# 每个tensor都有一个grad_fn属性，该属性即创建该tensor的Function,返回运算操作的对象(若无则返回None)

# gradient operation
# 直接创建的节点称为叶子节点，叶子节点对应的grad_fn为False

In [2]:
x = torch.ones(2, 2, requires_grad=True)
print(x)
print(x.grad_fn)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)
None


In [3]:
y = x + 2
print(y)
print(x.is_leaf, y.is_leaf)
print(y.grad_fn)

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)
True False


In [4]:
z = y * y * 3
out = z.mean()
print(z.is_leaf, out.is_leaf)
print(z, out)

False False
tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>) tensor(27., grad_fn=<MeanBackward0>)


In [5]:
# 使用 .requires_grad_ 来用inplace的方式改变requires_grad 属性
a = torch.randn(2, 2) # default requires_grad = False
a = ((a * 3) / (a - 1))
print(a.requires_grad) # False
a.requires_grad_(True)
print(a.requires_grad)
b = (a * a).sum()
print(b.grad_fn)

False
True


In [6]:
out.backward()
# x = [[1, 1], [1, 1]]
# grad = 1/4 * 3 * (1 + 2) * 1 = 9 / 2 = 4.5
print(x.grad)

# grad 在反向传播过程中是累加的
out2 = x.sum()
out2.backward() # 4.5 + 1 = 5.5
print(x.grad)

out3 = x.sum()
x.grad.data.zero_() # 梯度值设为0
out3.backward() # 0 + 1 = 1
print(x.grad)

tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])
tensor([[5.5000, 5.5000],
        [5.5000, 5.5000]])
tensor([[1., 1.],
        [1., 1.]])


**不允许张量对张量求导， 只允许标量对张良求导， 求导结果是和自变量同形的张量**, 必要时要把张量的通过所有张量的元素加权求和的方式转换为标量

In [7]:
# 在 y.backward()时， 如果y是标量， 则不需要为backward()传入任何参数; 否则，需要传入一个与y同形的shape
# y.backward（w）不是 y对x的导数， 而是 I = torch.sum(y*w) 对 x 的倒数
x = torch.tensor([1.0, 2.0, 3.0, 4.0], requires_grad=True)
y = 2 * x
z = y.view(2 , 2) # reshape
print(z) 

tensor([[2., 4.],
        [6., 8.]], grad_fn=<ViewBackward>)


In [8]:
# 现在z不是一个标量
# 传入与z同形的权重tensor进行加权求和得到一个标量
v = torch.tensor([[1.0, 0.1], [0.01, 0.001]], dtype=torch.float)
z.backward(v)
print(x.grad)

tensor([2.0000, 0.2000, 0.0200, 0.0020])


**中断梯度追踪**

In [9]:
import torch

# detach
x = torch.tensor(1.0, requires_grad=True)
y1 = x ** 2
with torch.no_grad():
    y2 = x ** 3
    
y3 = y1 + y2

print(x.requires_grad)
print(y1, y1.requires_grad) 
print(y2, y2.requires_grad)
print(y3, y3.requires_grad)


True
tensor(1., grad_fn=<PowBackward0>) True
tensor(1.) False
tensor(2., grad_fn=<AddBackward0>) True


In [10]:
y3.backward()
print(x.grad)

tensor(2.)


In [11]:
# 更改 tensor 数值， 不被autograde记录

x = torch.ones(1, requires_grad=True)

print(x.data)
print(x.data.requires_grad)

x.data *= 100 # 只改变值，不会记录在计算图，不影响梯度传播
y = 2 * x

y.backward()
print(x.grad)

tensor([1.])
False
tensor([2.])
